In [ ]:
# Import Library yang Diperlukan
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pandas as pd
import pickle
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import gradio as gr

In [ ]:
# Fungsi Preprocessing Teks
def preprocess_text(text):
    """
    Preprocessing teks:
    - Mengubah teks menjadi huruf kecil
    - Membersihkan whitespace berlebih
    - Menjaga teks sebagai paragraf utuh
    
    Args:
        text (str): Teks input.
        
    Returns:
        str: Teks yang telah diproses atau None jika tidak valid.
    """
    if not isinstance(text, str) or text.strip() == "":
        return None
    
    # Bersihkan teks
    text = text.lower().strip()
    text = re.sub(r'\s+', ' ', text)  # Ganti multiple whitespace dengan satu spasi
    return text

In [ ]:
# Fungsi Tokenisasi
def tokenize_text(texts, tokenizer, max_length=256):
    """
    Tokenisasi teks menggunakan tokenizer IndoBERT.
    
    Args:
        texts (list): Daftar teks yang akan di-tokenisasi.
        tokenizer: Tokenizer BERT.
        max_length (int): Panjang maksimum token.
        
    Returns:
        dict: Token hasil tokenisasi, termasuk input_ids dan attention_mask.
    """
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

In [ ]:
# Fungsi Ekstraksi Fitur Stylometric
def extract_stylometric_features(text):
    """
    Ekstraksi fitur stylometric dari teks:
    - Panjang kata rata-rata
    - Rasio kata unik
    - Rasio tanda baca
    - Panjang kalimat
    
    Args:
        text (str): Input teks.
        
    Returns:
        dict: Fitur stylometric.
    """
    # Basic features
    n_chars = len(text)
    n_words = len(text.split())
    avg_word_length = n_chars / n_words if n_words > 0 else 0
    unique_word_ratio = len(set(text.split())) / n_words if n_words > 0 else 0
    
    # Syntactic features
    punctuation_ratio = len(re.findall(r'[.,!?;:]', text)) / n_chars if n_chars > 0 else 0
    
    return {
        'avg_word_length': avg_word_length,
        'unique_word_ratio': unique_word_ratio,
        'punctuation_ratio': punctuation_ratio,
        'sentence_length': n_words
    }

In [ ]:
# Fungsi untuk Menghitung Similarity
def compute_similarity(embedding1, embedding2):
    """
    Menghitung cosine similarity antara dua embeddings.
    
    Args:
        embedding1: Embedding pertama.
        embedding2: Embedding kedua.
        
    Returns:
        float: Similarity score.
    """
    # Cosine similarity: dot product of normalized vectors
    return np.sum(embedding1 * embedding2, axis=1)

In [ ]:
def load_models(model_dir='saved_models'):
    """
    Load semua model dan konfigurasi yang diperlukan untuk inferensi.
    
    Args:
        model_dir (str): Direktori tempat model disimpan.
        
    Returns:
        tuple: Tokenizer, bi-encoder models, classifier, dan scaler.
    """
    print("Loading models and configurations...")
    
    # Load tokenizer
    tokenizer = BertTokenizer.from_pretrained(f'{model_dir}/tokenizer')
    
    # Register custom objects
    custom_objects = {'TFBertModel': TFBertModel}
    
    # Load bi-encoder models with custom objects
    bi_encoder_student_chatgpt = tf.keras.models.load_model(
        f'{model_dir}/bi_encoder_student_chatgpt.h5',
        custom_objects=custom_objects
    )
    
    bi_encoder_only_chatgpt = tf.keras.models.load_model(
        f'{model_dir}/bi_encoder_only_chatgpt.h5',
        custom_objects=custom_objects
    )
    
    # Load classifier with custom objects
    classifier = tf.keras.models.load_model(
        f'{model_dir}/text_classifier.h5',
        custom_objects=custom_objects
    )
    
    # Load scaler
    with open(f'{model_dir}/scaler_stylometric.pkl', 'rb') as f:
        scaler = pickle.load(f)
    
    # Load reference embeddings (jika ada)
    reference_embeddings = {}
    try:
        if os.path.exists(f'{model_dir}/reference_embeddings.pkl'):
            with open(f'{model_dir}/reference_embeddings.pkl', 'rb') as f:
                reference_embeddings = pickle.load(f)
        else:
            print("Warning: reference_embeddings.pkl not found. Using empty reference embeddings.")
    except Exception as e:
        print(f"Warning: Failed to load reference embeddings: {e}")
    
    print("Models and configurations loaded successfully!")
    return tokenizer, bi_encoder_student_chatgpt, bi_encoder_only_chatgpt, classifier, scaler, reference_embeddings


In [ ]:
# Fungsi Utama untuk Deteksi Teks
def detect_text_source(text, tokenizer, bi_encoder_student_chatgpt, bi_encoder_only_chatgpt, 
                       classifier, scaler, reference_embeddings=None):
    """
    Mendeteksi apakah teks berasal dari manusia atau AI.
    
    Args:
        text (str): Teks yang akan dideteksi.
        tokenizer: Tokenizer untuk preprocessing teks.
        bi_encoder_student_chatgpt: Model bi-encoder untuk Student_ChatGPT.
        bi_encoder_only_chatgpt: Model bi-encoder untuk Only_ChatGPT.
        classifier: Model klasifikasi.
        scaler: Scaler untuk normalisasi fitur stylometric.
        reference_embeddings: Embeddings referensi untuk similarity (opsional).
        
    Returns:
        dict: Hasil deteksi dan confidence score.
    """
    # Preprocessing teks
    processed_text = preprocess_text(text)
    if not processed_text:
        return {
            "source": "Error",
            "confidence": 0.0,
            "message": "Teks tidak valid atau kosong"
        }
    
    # Tokenisasi
    tokens = tokenize_text([processed_text], tokenizer)
    
    # Generate embeddings
    embedding_student_chatgpt = bi_encoder_student_chatgpt([tokens['input_ids'], tokens['attention_mask']])
    embedding_only_chatgpt = bi_encoder_only_chatgpt([tokens['input_ids'], tokens['attention_mask']])
    
    # Ekstraksi fitur stylometric
    stylometric_features = extract_stylometric_features(processed_text)
    stylometric_features_df = pd.DataFrame([stylometric_features])
    
    # Normalisasi fitur stylometric
    stylometric_features_normalized = scaler.transform(stylometric_features_df)
    
    # Hitung similarity scores dengan reference embeddings jika tersedia
    student_sim = 0.5
    chatgpt_sim = 0.5
    knowledge_sim = 0.5
    
    if reference_embeddings and 'student' in reference_embeddings and 'chatgpt_1' in reference_embeddings and 'chatgpt_2' in reference_embeddings:
        # Hitung similarity dengan student embeddings
        student_similarities = compute_similarity(
            embedding_student_chatgpt.numpy(), 
            reference_embeddings['student']
        )
        student_sim = np.mean(student_similarities)
        
        # Hitung similarity dengan chatgpt embeddings
        chatgpt_similarities = compute_similarity(
            embedding_student_chatgpt.numpy(), 
            reference_embeddings['chatgpt_1']
        )
        chatgpt_sim = np.mean(chatgpt_similarities)
        
        # Hitung similarity dengan knowledge embeddings
        knowledge_similarities = compute_similarity(
            embedding_only_chatgpt.numpy(), 
            reference_embeddings['chatgpt_2']
        )
        knowledge_sim = np.mean(knowledge_similarities)
    
    # Siapkan similarity scores untuk input model
    similarity_scores = np.array([[student_sim, chatgpt_sim, knowledge_sim]])
    
    # Prediksi menggunakan classifier
    prediction = classifier.predict({
        "bert_embedding": embedding_student_chatgpt.numpy(),
        "stylometric_features": stylometric_features_normalized,
        "similarity_score": similarity_scores
    })
    
    # Interpretasi hasil
    confidence = float(prediction[0][0])
    source = "AI (ChatGPT)" if confidence > 0.5 else "Human (Student)"
    
    return {
        "source": source,
        "confidence": confidence if source == "AI (ChatGPT)" else 1 - confidence,
        "stylometric_features": stylometric_features,
        "embedding_similarity": {
            "student_similarity": student_sim,
            "chatgpt_similarity": chatgpt_sim,
            "knowledge_similarity": knowledge_sim
        }
    }


In [ ]:
# Load Model dan Konfigurasi
try:
    tokenizer, bi_encoder_student_chatgpt, bi_encoder_only_chatgpt, classifier, scaler, reference_embeddings = load_models()
    models_loaded = True
    print("All models loaded successfully!")
except Exception as e:
    models_loaded = False
    print(f"Error loading models: {e}")
    print("Please make sure the models are available in the 'saved_models' directory.")

In [ ]:
# Fungsi untuk Visualisasi Hasil
def visualize_results(result):
    """
    Visualisasi hasil deteksi.
    
    Args:
        result (dict): Hasil deteksi dari fungsi detect_text_source.
    """
    if 'message' in result:
        print(f"Error: {result['message']}")
        return
    
    # Tampilkan hasil utama
    print(f"Detected Source: {result['source']}")
    print(f"Confidence: {result['confidence']:.4f}")
    
    # Visualisasi fitur stylometric
    features = result['stylometric_features']
    feature_names = list(features.keys())
    feature_values = list(features.values())
    
    plt.figure(figsize=(10, 6))
    plt.bar(feature_names, feature_values, color='skyblue')
    plt.title('Stylometric Features')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Visualisasi similarity scores
    similarities = result['embedding_similarity']
    sim_names = list(similarities.keys())
    sim_values = list(similarities.values())
    
    plt.figure(figsize=(10, 6))
    plt.bar(sim_names, sim_values, color='lightgreen')
    plt.title('Embedding Similarity Scores')
    plt.ylabel('Similarity')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
# Demo Inferensi dengan Contoh Teks
if models_loaded:
    # Contoh teks untuk dideteksi
    sample_texts = [
        "Saya menulis esai ini untuk tugas sekolah. Menurut pendapat saya, pemanasan global adalah masalah yang sangat serius dan kita semua harus bertindak sekarang. Saya pikir kita perlu lebih banyak kesadaran tentang masalah ini.",
        "Pemanasan global merupakan fenomena peningkatan suhu rata-rata permukaan bumi yang disebabkan oleh berbagai faktor, terutama aktivitas manusia yang menghasilkan gas rumah kaca. Dampaknya sangat signifikan terhadap ekosistem global dan kehidupan manusia. Berbagai penelitian ilmiah telah menunjukkan bahwa perubahan iklim yang terjadi saat ini berlangsung pada tingkat yang mengkhawatirkan."
    ]
    
    # Deteksi sumber teks
    for i, text in enumerate(sample_texts):
        print(f"\n{'='*50}")
        print(f"Sample Text #{i+1}:")
        print(f"{'='*50}")
        print(text[:100] + "..." if len(text) > 100 else text)
        print('-'*50)
        
        result = detect_text_source(
            text, 
            tokenizer, 
            bi_encoder_student_chatgpt, 
            bi_encoder_only_chatgpt, 
            classifier, 
            scaler,
            reference_embeddings
        )
        
        visualize_results(result)

In [ ]:
# Fungsi untuk Aplikasi Web dengan Gradio
def predict_source_for_gradio(text):
    """
    Fungsi untuk memprediksi sumber teks untuk aplikasi web Gradio.
    
    Args:
        text (str): Teks input.
        
    Returns:
        tuple: Label, confidence, dan visualisasi.
    """
    if not text.strip():
        return "Masukkan teks terlebih dahulu", 0.0, None, None
    
    result = detect_text_source(
        text, 
        tokenizer, 
        bi_encoder_student_chatgpt, 
        bi_encoder_only_chatgpt, 
        classifier, 
        scaler,
        reference_embeddings
    )
    
    if 'message' in result:
        return result['message'], 0.0, None, None
    
    # Buat visualisasi fitur stylometric
    features = result['stylometric_features']
    feature_df = pd.DataFrame({
        'Feature': list(features.keys()),
        'Value': list(features.values())
    })
    fig1 = plt.figure(figsize=(8, 5))
    sns.barplot(x='Feature', y='Value', data=feature_df)
    plt.title('Stylometric Features')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Buat visualisasi similarity scores
    similarities = result['embedding_similarity']
    sim_df = pd.DataFrame({
        'Type': list(similarities.keys()),
        'Similarity': list(similarities.values())
    })
    fig2 = plt.figure(figsize=(8, 5))
    sns.barplot(x='Type', y='Similarity', data=sim_df)
    plt.title('Embedding Similarity Scores')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    return result['source'], result['confidence'], fig1, fig2

In [ ]:
# Jalankan Aplikasi Web dengan Gradio (jika models_loaded)
if models_loaded:
    # Buat interface Gradio
    iface = gr.Interface(
        fn=predict_source_for_gradio,
        inputs=gr.Textbox(lines=10, placeholder="Masukkan teks di sini..."),
        outputs=[
            gr.Label(label="Sumber Teks"),
            gr.Number(label="Confidence Score"),
            gr.Plot(label="Fitur Stylometric"),
            gr.Plot(label="Similarity Scores")
        ],
        title="Deteksi Teks AI vs Manusia",
        description="Aplikasi ini mendeteksi apakah teks ditulis oleh manusia (siswa) atau dihasilkan oleh AI (ChatGPT).",
        examples=[
            ["Saya menulis esai ini untuk tugas sekolah. Menurut pendapat saya, pemanasan global adalah masalah yang sangat serius dan kita semua harus bertindak sekarang."],
            ["Pemanasan global merupakan fenomena peningkatan suhu rata-rata permukaan bumi yang disebabkan oleh berbagai faktor, terutama aktivitas manusia yang menghasilkan gas rumah kaca. Dampaknya sangat signifikan terhadap ekosistem global."]
        ]
    )
    
    # Jalankan aplikasi
    iface.launch(share=True)  # share=True untuk membuat link publik


In [ ]:
# Fungsi untuk Inferensi Batch (untuk file CSV/Excel)
def batch_inference(file_path, output_path=None):
    """
    Melakukan inferensi batch pada file CSV atau Excel.
    
    Args:
        file_path (str): Path ke file CSV atau Excel.
        output_path (str, optional): Path untuk menyimpan hasil. Default None.
        
    Returns:
        pd.DataFrame: DataFrame hasil deteksi.
    """
    if not models_loaded:
        print("Models not loaded. Please load models first.")
        return None
    
    # Load file
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith(('.xlsx', '.xls')):
        df = pd.read_excel(file_path)
    else:
        print("Unsupported file format. Please use CSV or Excel.")
        return None
    
    # Cek kolom teks
    text_column = None
    possible_columns = ['text', 'content', 'essay', 'teks', 'konten', 'esai']
    for col in possible_columns:
        if col in df.columns:
            text_column = col
            break
    
    if text_column is None and len(df.columns) > 0:
        # Gunakan kolom pertama jika tidak ada kolom yang cocok
        text_column = df.columns[0]
    
    if text_column is None:
        print("No suitable text column found in the file.")
        return None
    
    # Tambahkan kolom hasil
    results = []
    
    # Proses setiap baris
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
        text = row[text_column]
        result = detect_text_source(
            text, 
            tokenizer, 
            bi_encoder_student_chatgpt, 
            bi_encoder_only_chatgpt, 
            classifier, 
            scaler,
            reference_embeddings
        )
        
        # Tambahkan hasil ke list
        results.append({
            'source': result.get('source', 'Error'),
            'confidence': result.get('confidence', 0.0),
            'avg_word_length': result.get('stylometric_features', {}).get('avg_word_length', 0),
            'unique_word_ratio': result.get('stylometric_features', {}).get('unique_word_ratio', 0),
            'punctuation_ratio': result.get('stylometric_features', {}).get('punctuation_ratio', 0),
            'sentence_length': result.get('stylometric_features', {}).get('sentence_length', 0),
            'student_similarity': result.get('embedding_similarity', {}).get('student_similarity', 0),
            'chatgpt_similarity': result.get('embedding_similarity', {}).get('chatgpt_similarity', 0),
            'knowledge_similarity': result.get('embedding_similarity', {}).get('knowledge_similarity', 0)
        })
    
    # Buat DataFrame hasil
    results_df = pd.DataFrame(results)
    
    # Gabungkan dengan DataFrame asli
    output_df = pd.concat([df, results_df], axis=1)
    
    # Simpan hasil jika output_path diberikan
    if output_path:
        if output_path.endswith('.csv'):
            output_df.to_csv(output_path, index=False)
        elif output_path.endswith(('.xlsx', '.xls')):
            output_df.to_excel(output_path, index=False)
        else:
            output_df.to_csv(output_path + '.csv', index=False)
        print(f"Results saved to {output_path}")
    
    return output_df

# %%
# Demo Batch Inference (uncomment untuk menggunakan)
"""
if models_loaded:
    # Contoh penggunaan batch inference
    from tqdm import tqdm
    
    # Path ke file input
    input_file = "sample_essays.csv"
    
    # Path untuk menyimpan hasil
    output_file = "detection_results.csv"
    
    # Jalankan batch inference
    results = batch_inference(input_file, output_file)
    
    # Tampilkan ringkasan hasil
    if results is not None:
        print("\nSummary of Results:")
        print(f"Total texts processed: {len(results)}")
        print(f"Detected as Human: {sum(results['source'] == 'Human (Student)')}")
        print(f"Detected as AI: {sum(results['source'] == 'AI (ChatGPT)')}")
        print(f"Average confidence: {results['confidence'].mean():.4f}")
        
        # Visualisasi distribusi confidence
        plt.figure(figsize=(10, 6))
        sns.histplot(results['confidence'], bins=20, kde=True)
        plt.title('Distribution of Confidence Scores')
        plt.xlabel('Confidence')
        plt.ylabel('Count')
        plt.show()
"""

# %%
# Fungsi untuk Inferensi Interaktif di Notebook
def interactive_inference():
    """
    Fungsi untuk inferensi interaktif di notebook.
    """
    if not models_loaded:
        print("Models not loaded. Please load models first.")
        return
    
    # Input teks dari pengguna
    text = input("Masukkan teks untuk dideteksi (ketik 'exit' untuk keluar):\n")
    
    while text.lower() != 'exit':
        # Deteksi sumber teks
        result = detect_text_source(
            text, 
            tokenizer, 
            bi_encoder_student_chatgpt, 
            bi_encoder_only_chatgpt, 
            classifier, 
            scaler,
            reference_embeddings
        )
        
        # Tampilkan hasil
        print("\nHasil Deteksi:")
        print(f"Sumber: {result.get('source', 'Error')}")
        print(f"Confidence: {result.get('confidence', 0):.4f}")
        
        # Tampilkan fitur stylometric
        print("\nFitur Stylometric:")
        for feature, value in result.get('stylometric_features', {}).items():
            print(f"  - {feature}: {value:.4f}")
        
        # Tampilkan similarity scores
        print("\nSimilarity Scores:")
        for sim_type, value in result.get('embedding_similarity', {}).items():
            print(f"  - {sim_type}: {value:.4f}")
        
        # Input teks berikutnya
        print("\n" + "="*50)
        text = input("Masukkan teks untuk dideteksi (ketik 'exit' untuk keluar):\n")

In [ ]:
if models_loaded:
    interactive_inference()


In [ ]:
print("Script started")
print(f"Models loaded successfully: {models_loaded}")

# At the end of the script
if models_loaded:
    print("Running interactive inference...")
    interactive_inference()
else:
    print("Models failed to load. Check the 'saved_models' directory.")
